# Advection-Diffusion Equation
## Ref - Perturbation Methods by Hinch Sec. 7.3

### Author - Pratik Aghor

* We consider an advection diffusion equation with strong advection and weak diffusion.
$\frac{\partial f}{\partial t} + \frac{\partial}{\partial \theta} (\omega(\theta)f) = \epsilon \frac{\partial^{2} f}{\partial \theta^{2}}$

with $t \geq 0$, $0\leq \theta \leq 2\pi$ and $f(\theta, t=0) = F(\theta)$

In [1]:
function advection_diffusion_integrate(f, ω, ϵ, Lx, dt, Nt, nsave)
    
    Nx = length(f)                  # number of gridpoints
    x = collect(0:(Nx-1)/Nx)*Lx
    kx = vcat(0:Nx/2-1, 0, -Nx/2+1:-1)  # integer wavenumbers: exp(2*pi*kx*x/L)
    @show kx
    alpha = 2*pi*kx/Lx              # real wavenumbers:    exp(alpha*x)
    D = 1im*alpha;                  # D = d/dx operator in Fourier space

    # Express PDE as u_t = Lu + N(u), L is linear part, N nonlinear part.
    # Then Crank-Nicolson Adams-Bashforth discretization is 
    # 
    # (I - dt/2 L) u^{n+1} = (I + dt/2 L) u^n + 3dt/2 N^n - dt/2 N^{n-1}
    #
    # let A = (I - dt/2 L) 
    #     B = (I + dt/2 L), then the CNAB timestep formula is
    # 
    # u^{n+1} = A^{-1} (B u^n + 3dt/2 N^n - dt/2 N^{n-1}) 

    L = -ϵ.*(alpha.^2)        # linear operator \epsilon*D^2 in Fourier space
    
    
    Nsave = div(Nt, nsave)+1        # number of saved time steps, including t=0
    t = (0:Nsave)*(dt*nsave)        # t timesteps
    U = zeros(Nsave, Nx)            # matrix of u(xⱼ, tᵢ) values
    U[1,:] = f                      # assign initial condition to U
    s = 2                           # counter for saved data
    
    dt2  = dt/2
    dt32 = 3*dt/2;
    A_inv = (ones(Nx) - dt2*L).^(-1)
    B     =  ones(Nx) + dt2*L

    Nn  = -fft(ω.*f)    # -(\omega u)_theta (spectral), notation Nn = N^n     = N(u(n dt))
    Nn1 = copy(Nn)     #                   notation Nn1 = N^{n-1} = N(u((n-1) dt))
    v  = fft(f)        # transform u to spectral

    # timestepping loop
    for n = 1:Nt
        Nn1 = copy(Nn)                 # shift nonlinear term in time: N^{n-1} <- N^n
        Nn  = -fft(real(ifft(v)).*ω) # compute Nn = -u u_x

        v = A_inv .* (B .* v + dt32*Nn - dt2*Nn1)
        
        if mod(n, nsave) == 0
            U[s,:] = real(ifft(v))
            s += 1            
        end
    end

    t,U
end

advection_diffusion_integrate (generic function with 1 method)

In [4]:
using FFTW

Lx= 2*pi
Nx = 256
dt = 1/16
nsave = 1
Nt = 1600
ϵ = 0.001

θ = Lx*(0:Nx-1)/Nx
ω = 2*ones(Nx) + sin.(θ)
#IC 
F = ones(Nx) + sin.(θ)
f = F
t,U = advection_diffusion_integrate(f, ω, ϵ, Lx, dt, Nt, nsave)


kx = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 0.0, -127.0, -126.0, -125.0, -124.0, -123.0, -122.0, -121.0, -120.0, -119.0, -118.0, -117.0, -116.0, -115.0, -114.0, -113.0, -112.0, -111.0, -110.0, -109.0, -108.0, -107.0, -106.0, -105.0, -104.0, -103.0, -10

(0.0:0.0625:100.0625, [1.0 1.02454 … 0.950932 0.975459; 0.874996 0.894897 … 0.834981 0.85502; … ; 1.09266e-33 1.10775e-33 … 2.1228e-33 1.91678e-34; 1.44694e-33 -1.89003e-35 … 1.44683e-33 -3.6476e-34])

In [5]:
using Plots
anim = Animation()

@gif for i in 1:(Int64)(Nt/nsave)
    p = plot(θ, U[i, :])
    plot!(p[1], legend=false, xlabel="theta", ylabel="u")
    frame(anim)
end
gif(anim, "/home/aghor/Aghor/UNH/summer_2019/Chini/anim_fps2.gif")

┌ Info: Saved animation to 
│   fn = /home/aghor/Aghor/UNH/summer_2019/Chini/tmp.gif
└ @ Plots /home/aghor/.julia/packages/Plots/47Tik/src/animation.jl:90
┌ Info: Saved animation to 
│   fn = /home/aghor/Aghor/UNH/summer_2019/Chini/anim_fps2.gif
└ @ Plots /home/aghor/.julia/packages/Plots/47Tik/src/animation.jl:90


Plots.AnimatedGif("/home/aghor/Aghor/UNH/summer_2019/Chini/anim_fps2.gif")